In [1]:
import numpy as np
from scipy import linalg as LA
from operator import itemgetter
import time

In [6]:
class PageRank:
  '''Class used to find the page ranks
    1. with random teleportations
    2. without random teleportations
  using the following techniques to find the principle left eigenvector-
    1. using the numerical linear algebra package
    2. using the Power iteration method

    Class variables:
      n: number of nodes in the web graph
      e: number of edges/connection in the web graph
      alpha: hyperparameter indicating probability of random teleportation
      adj: adjacency matrix
      edges: list containing tuples indicating the start and end point of the connections

      PT: probability transition matrix for case without random teleportations
      PT_tele: probability transition matrix for case with random teleportations
      left_eig1: pr left eigenvector for PT_tele found using lin alg library
      left_eig2: pr left eigenvector for PT_tele found using power iteration
      left_eig3: pr left eigenvector for PT found using lin alg library
      left_eig4: pr left eigenvector for PT found using power iteration
  '''
  def __init__(self):
    '''Function for initializing and taking input necessary for the algorithm'''

    # inputting n and e
    self.n = int(input("Pls enter the number of nodes: "))
    self.e = int(input("Pls enter the number of edges: "))
    # self.n = 3

    self.alpha = 0.1

    self.adj = np.zeros((self.n, self.n))
    self.edges = []


    self.left_eig1 = []
    self.left_eig2 = []
    self.left_eig3 = []
    self.left_eig4 = []

    self.iter1 = 0
    self.iter2 = 0

    # Sample test case
    # self.edges = [(1,2), (2,1), (1,3), (3,1)]
    # for edge in self.edges:
    #   s = edge[0]
    #   d = edge[1]
    #   self.adj[s-1][d-1] = 1

    # inputting the connections from the user and adjusting adj matrix
    print("Enter the start and end nodes of each edge: \n")
    for i in range(self.e):
      s, e = [int(x) for x in input().split(',')]
      self.edges.append((s-1,e-1))
      self.adj[s-1][e-1] = 1

    # print("Adjacency matrix:\n")
    # for row in self.adj:
    #   print(row)
    # print("\n")
    
    # initializing the probability transition matrices to be of size n*n and contain 0s
    self.PT_tele = np.zeros((self.n,self.n), dtype=float)
    self.PT = np.zeros((self.n,self.n), dtype=float)

  def calc_PT(self):
    '''Function to find PT'''
    for row in range(self.n):
      s = sum(self.adj[row])
      for col in range(self.n):
        if(s!=0):
          self.PT[row][col] = self.adj[row][col]/s
    self.PT = self.PT +(1-0.999999)/self.n*np.ones([self.n,self.n])

  def calc_PT_tele(self):
    '''Function to find PT_tele'''
    for row in range(self.n):
      s = sum(self.adj[row])
      # print(s)
      for col in range(self.n):
        if self.adj[row][col] == 0:
          if (self.n-s != 0):
            self.PT_tele[row][col] = (1-self.alpha)/(self.n)
        else:
          if(s!=0):
            self.PT_tele[row][col] = self.alpha/s + (1-self.alpha)/(self.n)
  
  def using_alg_package(self, prob_mat):
    '''
    Finding the principle left eigen vector for prob_mat using lin alg library
    Parameter:
      prob_mat: n*n probability transition matrix for which we need to find the pr left eigen vector
    Return value:
      left_eig: list representing the pr left eigen vector
    '''
    v, V= LA.eig(a=np.array(prob_mat),b=None,left=True,right=False,overwrite_a=False,overwrite_b=False,check_finite=False)
    # v, V = LA.eig(prob_mat.T)
    max_eig_val = 0
    max_eig_val_ind = 0
    for i in range(len(v)):
      if (v[max_eig_val_ind] < v[i]):
        max_eig_val_ind = i
        max_eig_val = v[i]

    left_eig = V[:, max_eig_val_ind]
    left_eig = left_eig/LA.norm(left_eig)
    return left_eig

  def power_iter(self, prob_mat, iter=1000):
    '''
    Finding the principle left eigen vector for prob_mat using Power Iteration
    Parameter:
      prob_mat: n*n probability transition matrix for which we need to find the pr left eigen vector
    Return value:
      left_eig: list representing the pr left eigen vector
    '''

    left_eig = np.ones(self.n)
    # left_eig[0] = 1
    left_eig_prev = left_eig.copy()
    for i in range(iter):
      left_eig = np.dot(left_eig_prev, prob_mat)
      left_eig = left_eig/LA.norm(left_eig)
      # norm = LA.norm(left_eig - left_eig_prev)
      # if (norm < 0.0000000001):
      #   return i+1, left_eig
      left_eig_prev = left_eig
    return iter, left_eig
  
  def print_res(self, page_ranks):
    result = []
    for i in range(self.n):
      result.append((i+1, page_ranks[i]))

    result=sorted(result,key=itemgetter(1),reverse=True)

    print("Page Rankings:\nRank\tPage No\tProbablity")
    index=1
    for x in result:
      print(index,end="\t")
      for j in x:
        print(j,end="\t")
      index=index+1
      print("\n")

  def run(self):
    '''Function to be called in order to run the Page Rank algorithm'''
    start = time.time()

    self.calc_PT()
    self.calc_PT_tele()

    # sample test case
    # self.PT = np.array([[1/6, 2/3, 1/6], [5/12, 1/6, 5/12], [1/6, 2/3, 1/6]])

    self.left_eig1 = self.using_alg_package(self.PT_tele)
    iter1, self.left_eig2 = self.power_iter(self.PT_tele)
    self.left_eig3 = self.using_alg_package(self.PT)
    iter2, self.left_eig4 = self.power_iter(self.PT)

    end = time.time()

    print("Runtime: ", end-start)

    print("\n\n")
    print("Probability transition matrix with random teleportations:\n", self.PT_tele)
    print("Probability transition matrix without random teleportations:\n", self.PT)

    print("\n")

    print("PRINCIPLE LEFT EIGEN VECTOR")
    print("1. WITH RANDOM TELEPOTATIONS:\n")
  
    print("a. Using linear algebra packages:")
    # print("\t\t", self.left_eig1)
    self.print_res(self.left_eig1)
    # print("Sum = ", sum(self.left_eig1))

    print("b. Using Power Iteration method:")
    # print("\t\t", self.left_eig2)
    self.print_res(self.left_eig2)
    # print("Sum = ", sum(self.left_eig2))
    # print("No. of iterations = ", iter1)

    print("\n")

    print("2. WITHOUT RANDOM TELEPOTATIONS:\n")

    print("a. Using linear algebra packages:")
    # print("\t\t", self.left_eig3)
    self.print_res(self.left_eig3)
    # print("Sum = ", sum(self.left_eig3))

    print("a. Using Power Iteration method:")
    # print("\t\t", self.left_eig4)
    self.print_res(self.left_eig4)
    # print("Sum = ", sum(self.left_eig4))
    # print("No. of iterations = ", iter2)

In [14]:
# Driver Code1,
page_rank = PageRank()
page_rank.run()

Pls enter the number of nodes: 4
Pls enter the number of edges: 5
Enter the start and end nodes of each edge: 

1,2
3,4
3,2
4,3
4,2
Runtime:  0.019129037857055664



Probability transition matrix with random teleportations:
 [[0.225 0.325 0.225 0.225]
 [0.225 0.225 0.225 0.225]
 [0.225 0.275 0.225 0.275]
 [0.225 0.275 0.275 0.225]]
Probability transition matrix without random teleportations:
 [[2.50000000e-07 1.00000025e+00 2.50000000e-07 2.50000000e-07]
 [2.50000000e-07 2.50000000e-07 2.50000000e-07 2.50000000e-07]
 [2.50000000e-07 5.00000250e-01 2.50000000e-07 5.00000250e-01]
 [2.50000000e-07 5.00000250e-01 5.00000250e-01 2.50000000e-07]]


PRINCIPLE LEFT EIGEN VECTOR
1. WITH RANDOM TELEPOTATIONS:

a. Using linear algebra packages:
Page Rankings:
Rank	Page No	Probablity
1	1	-0.46174822024789264	

2	3	-0.48678992901008744	

3	4	-0.48678992901008744	

4	2	-0.5593386372552962	

b. Using Power Iteration method:
Page Rankings:
Rank	Page No	Probablity
1	2	0.559338637255296	

2	3	0.48678992